### Task: Obtain the total goals and shots by a particular forward player

### Import the libraries

In [2]:
import pandas as pd
from pandas import DataFrame
from pymongo import MongoClient
import matplotlib.pyplot as plt

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Set up the MongoDB connection in Python

In [4]:
client=MongoClient('localhost',27017)
db=client['Wyscout']

In [5]:
forward_totalGoalShots=db.player_advance_stats.aggregate([
{'$unwind': "$positions"},
{'$unwind': "$total"},
{'$project':{"_id":0,"playerId":1,"competitionId":1,"seasonId":1,"positions.position.code":1,"positions.position.name":1,"total.goals":1, "total.shots":1}
},
{'$match' : { '$or': [{"positions.position.code":"lwf"},{"positions.position.code":"rwf"},{"positions.position.code":"rw"},{"positions.position.code":"lw"},{"positions.position.code":"ss"},{"positions.position.code":"cf"}]} 
}
  
])
forward_totalGoalShots_df=pd.DataFrame(forward_totalGoalShots)

In [6]:
forward_totalGoalShots_df.head(5)

,playerId,competitionId,seasonId,positions,total
0,25413,364,185727,"{'position': {'name': 'Striker', 'code': 'cf'}}","{'goals': 7, 'shots': 37}"
1,25867,364,185727,"{'position': {'name': 'Striker', 'code': 'cf'}}","{'goals': 17, 'shots': 71}"
2,247671,364,185727,"{'position': {'name': 'Right Wing Forward', 'c...","{'goals': 5, 'shots': 41}"
3,247671,364,185727,"{'position': {'name': 'Striker', 'code': 'cf'}}","{'goals': 5, 'shots': 41}"
4,346158,364,185727,"{'position': {'name': 'Striker', 'code': 'cf'}}","{'goals': 1, 'shots': 9}"


### select only the required columns

In [7]:
forward_totalGoalShots_df1 = forward_totalGoalShots_df[['playerId','competitionId','seasonId','total']]

In [8]:
forward_totalGoalShots_df1.head(5)

,playerId,competitionId,seasonId,total
0,25413,364,185727,"{'goals': 7, 'shots': 37}"
1,25867,364,185727,"{'goals': 17, 'shots': 71}"
2,247671,364,185727,"{'goals': 5, 'shots': 41}"
3,247671,364,185727,"{'goals': 5, 'shots': 41}"
4,346158,364,185727,"{'goals': 1, 'shots': 9}"


### split the 'total' dict into separate columns 'goals' & 'shots'  

In [9]:
forward_totalGoalShots_df1 = pd.concat([forward_totalGoalShots_df1.drop(['total'], axis=1), pd.DataFrame(forward_totalGoalShots_df1['total'].to_list())], axis=1)

In [10]:
forward_totalGoalShots_df1.head(5)

,playerId,competitionId,seasonId,goals,shots
0,25413,364,185727,7,37
1,25867,364,185727,17,71
2,247671,364,185727,5,41
3,247671,364,185727,5,41
4,346158,364,185727,1,9


### Remove duplicate rows from  "forward_totalGoalShots_df1"

In [11]:
forward_totalGoalShots_df5=forward_totalGoalShots_df1.drop_duplicates()
forward_totalGoalShots_df5.head(5)

,playerId,competitionId,seasonId,goals,shots
0,25413,364,185727,7,37
1,25867,364,185727,17,71
2,247671,364,185727,5,41
4,346158,364,185727,1,9
5,3360,364,185727,1,21


### Read matches_events & Create a DF

In [12]:
event_details=db.matches_events_scaling_v02.find({}, {"_id":0 })
event_details_df=pd.DataFrame(event_details)

In [13]:
event_details_df.to_csv('event_details_df.csv')

In [14]:
event_details_df = pd.read_csv('event_details_df.csv', index_col=0)

In [17]:
event_details_df=event_details_df.drop_duplicates()
event_details_df.head(5)

,index,id,playerId,teamId,matchId,matchPeriod,eventSec,eventId,eventName,subEventId,...,y start,x stop,y stop,tags,match_id,team_id,x start length (meters),y start length (meters),x stop length (meters),y stop length(meters)
0,0,613566210,346158,1609,2829960,1H,0.811034,8,Pass,85,...,51,28.0,63.0,1801,2829960,1609,52.50,34.68,29.40,42.84
1,1,613566221,20612,1609,2829960,1H,3.969.135,8,Pass,83,...,63,72.0,6.0,1801,2829960,1609,29.40,42.84,75.60,4.08
2,2,613566223,25867,1609,2829960,1H,7.277.111,7,Others on the ball,72,...,6,77.0,0.0,1302,2829960,1609,75.60,4.08,80.85,0.00
3,3,613565944,0,1625,2829960,1H,9.668.171,5,Interruption,50,...,100,NaN,NaN,NaN,2829960,1609,24.15,68.00,0.00,0.00
4,4,613565945,8277,1625,2829960,1H,16.161.527,3,Free Kick,36,...,100,3.0,50.0,1801,2829960,1609,17.85,68.00,3.15,34.00


### Merge matches_events and Forwards DF on playerId : Final Result

In [18]:
ForwardEventDetails=forward_totalGoalShots_df5.merge(event_details_df, on='playerId', how = 'left')

In [19]:
ForwardEventDetails.head(5)  # for playerId = 481911, there are no records in matches_events table, thats why 'NaN'

,playerId,competitionId,seasonId,goals,shots,index,id,teamId,matchId,matchPeriod,...,y start,x stop,y stop,tags,match_id,team_id,x start length (meters),y start length (meters),x stop length (meters),y stop length(meters)
0,25413,364,185727,7,37,1266.0,613567906.0,1609.0,2829960.0,2H,...,61.0,29.0,46.0,1801,2829960.0,1609.0,39.90,41.48,30.45,31.28
1,25413,364,185727,7,37,1291.0,613568133.0,1609.0,2829960.0,2H,...,74.0,37.0,87.0,"5,04,70,11,802",2829960.0,1609.0,35.70,50.32,38.85,59.16
2,25413,364,185727,7,37,1341.0,613567939.0,1609.0,2829960.0,2H,...,9.0,35.0,9.0,"70,31,801",2829960.0,1609.0,37.80,6.12,36.75,6.12
3,25413,364,185727,7,37,1351.0,613567949.0,1609.0,2829960.0,2H,...,43.0,22.0,29.0,"1,40,11,801",2829960.0,1609.0,26.25,29.24,23.10,19.72
4,25413,364,185727,7,37,1417.0,613567995.0,1609.0,2829960.0,2H,...,17.0,75.0,25.0,1802,2829960.0,1609.0,76.65,11.56,78.75,17.00


####  Just verifying few columns for NaN(ignore): for "playerId: 481911" there are no records in matches_events table  :

In [20]:
findplayerId= db.matches_events_scaling_v02.find(
    { "playerId": 481911}
)

In [21]:
findplayerId_df=pd.DataFrame(findplayerId)

In [22]:
type(findplayerId_df)
len(findplayerId_df)    # no records.

pandas.core.frame.DataFrame

0